In [1]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
set_session(sess)

C:\Users\James_Dev_Account\anaconda3\envs\honours-keras-2.1.4\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\James_Dev_Account\anaconda3\envs\honours-keras-2.1.4\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\James_Dev_Account\anaconda3\envs\honours-keras-2.1.4\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 

In [2]:
"""

This is for morphological classification of galaxies by CNN,
New regresssion for B/D ratio
By Kenji Bekki, on 2018/3/30

"""


import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import np_utils
from keras.models import model_from_json
### Added 2018/3/30
from keras.applications import imagenet_utils
from keras.models import load_model
###
import keras.callbacks
import numpy as np
#import keras.backend.tensorflow_backend as KTF
#import tensorflow as tf
import os.path


### Total model number = (100*1) * nmodel

#iset=int(input('Input the total number of sets of models '))
#nmodel0=int(input('Input the total number of images per model '))
#nmodel=nmodel0*iset
#epochs=int(input('Input the number of epochs'))
iset=5
nmodel0=100
epochs=1000
nmodel=nmodel0*iset
nmodel=1000
print('nmodel',nmodel)

### Original values
#batch_size = 128
#num_classes = 10
#epochs = 12
batch_size = 200
num_classes = 2
#epochs = 500
nb_epoch=epochs
n_mesh=50
#n_mesh=20
#nmodel=4000

img_rows, img_cols = n_mesh, n_mesh
n_mesh2=n_mesh*n_mesh-1
n_mesh3=n_mesh*n_mesh


print(img_rows, img_cols, n_mesh2)
#stop



#(x_train, y_train), (x_test, y_test) = mnist.load_data()
#x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
#x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

input_shape = (img_rows, img_cols, 1)

#x_train = x_train.astype('float32')
#x_test = x_test.astype('float32')
#x_train /= 255
#x_test /= 255
#print('x_train shape:', x_train.shape)
#print(x_train.shape[0], 'train samples')
#print(x_test.shape[0], 'test samples')
#print(y_test.shape[0], 'y.test samples')
#print(str(y_test[0]))
#print(str(y_test[1]))
#print(str(y_test[2]))

#y_train = y_train.astype('int32')
#y_test = y_test.astype('int32')
#y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
#y_test =  keras.utils.np_utils.to_categorical(y_test, num_classes)

# This is for simlation data sets


nmodel 1000
50 50 2499


In [3]:
from sklearn import model_selection

x_dataset_series = np.genfromtxt('2dft.dat', autostrip=True, max_rows=nmodel*n_mesh3)
y_dataset1 = np.genfromtxt('2dftn1.dat', autostrip=True, max_rows=nmodel)
y_dataset2 = np.genfromtxt('2dftn2.dat', autostrip=True, max_rows=nmodel)

x_dataset = x_dataset_series.reshape(nmodel, img_rows, img_cols, 1)
y_dataset = np.c_[y_dataset1, y_dataset2]

print(x_dataset.shape)
print(y_dataset.shape)

#x_train = x_dataset
#x_test = x_dataset

#y_train = y_dataset
#y_test = y_dataset

(1000, 50, 50, 1)
(1000, 2)


In [4]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(x_dataset, y_dataset, test_size=0.2)

In [5]:
print('Galaxy type',y_train[:5])

#stop

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

### For labelling of morphological types

#model.add(Dense(num_classes, activation='softmax'))
#model.compile(loss=keras.losses.categorical_crossentropy,
#              optimizer=keras.optimizers.Adadelta(),
#              metrics=['accuracy'])

### For regression of B/D

model.add(Dense(2, activation='linear'))
#model.add(Dense(1))
#model.add(Activation=('linear'))
#model.add(activation=('linear'))
model.compile(loss='mean_squared_error',
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
          verbose=1, validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])


print('save the architecture of a model')

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

- val_acc: 0.7400
Epoch 857/1000
800/800 [==============================] - 0s 196us/step - loss: 3.1375 - acc: 0.7900 - val_loss: 1.9110 - val_acc: 0.8200
Epoch 858/1000
800/800 [==============================] - 0s 197us/step - loss: 2.3487 - acc: 0.8462 - val_loss: 1.6560 - val_acc: 0.8300
Epoch 859/1000
800/800 [==============================] - 0s 199us/step - loss: 2.0881 - acc: 0.8675 - val_loss: 1.6015 - val_acc: 0.8350
Epoch 860/1000
800/800 [==============================] - 0s 196us/step - loss: 2.2338 - acc: 0.8488 - val_loss: 1.6645 - val_acc: 0.8250
Epoch 861/1000
800/800 [==============================] - 0s 199us/step - loss: 1.9487 - acc: 0.8600 - val_loss: 1.3360 - val_acc: 0.8500
Epoch 862/1000
800/800 [==============================] - 0s 193us/step - loss: 2.2154 - acc: 0.8625 - val_loss: 1.3943 - val_acc: 0.8600
Epoch 863/1000
800/800 [==============================] - 0s 192us/step - loss: 2.1079 - acc: 0.8600 - val_loss: 1.8202 - val_acc: 0.8350
Epoch 864/1000
8